# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f77049aaa30>
├── 'a' --> tensor([[ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498]])
└── 'x' --> <FastTreeValue 0x7f77049aad30>
    └── 'c' --> tensor([[ 0.0510, -1.4767, -0.5923,  0.1371],
                        [ 0.2061, -1.9645,  1.1369,  1.2084],
                        [ 1.0340, -0.8003,  0.6450,  0.1957]])

In [4]:
t.a

tensor([[ 0.5464, -0.1315, -0.6195],
        [ 0.2436,  1.2652,  0.5498]])

In [5]:
%timeit t.a

64.5 ns ± 0.0653 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f77049aaa30>
├── 'a' --> tensor([[-2.5174,  0.4183, -1.0256],
│                   [ 2.8935,  0.9359,  0.6651]])
└── 'x' --> <FastTreeValue 0x7f77049aad30>
    └── 'c' --> tensor([[ 0.0510, -1.4767, -0.5923,  0.1371],
                        [ 0.2061, -1.9645,  1.1369,  1.2084],
                        [ 1.0340, -0.8003,  0.6450,  0.1957]])

In [7]:
%timeit t.a = new_value

59.5 ns ± 0.0184 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498]]),
    x: Batch(
           c: tensor([[ 0.0510, -1.4767, -0.5923,  0.1371],
                      [ 0.2061, -1.9645,  1.1369,  1.2084],
                      [ 1.0340, -0.8003,  0.6450,  0.1957]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5464, -0.1315, -0.6195],
        [ 0.2436,  1.2652,  0.5498]])

In [11]:
%timeit b.a

52.8 ns ± 0.0382 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2854,  0.3197,  0.6126],
               [ 0.4510, -0.1710, -1.4781]]),
    x: Batch(
           c: tensor([[ 0.0510, -1.4767, -0.5923,  0.1371],
                      [ 0.2061, -1.9645,  1.1369,  1.2084],
                      [ 1.0340, -0.8003,  0.6450,  0.1957]]),
       ),
)

In [13]:
%timeit b.a = new_value

489 ns ± 1.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

853 ns ± 9.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 28.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

140 µs ± 414 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 335 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f77049b01c0>
├── 'a' --> tensor([[[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]],
│           
│                   [[ 0.5464, -0.1315, -0.6195],
│                    [ 0.2436,  1.2652,  0.5498]]])
└── 'x' --> <FastTreeValue 0x7f77049b0340>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.4 µs ± 75.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f764ec56d30>
├── 'a' --> tensor([[ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498],
│                   [ 0.5464, -0.1315, -0.6195],
│                   [ 0.2436,  1.2652,  0.5498]])
└── 'x' --> <FastTreeValue 0x7f764ec56d00>
    └── 'c' --> tensor([[ 0.0510, -1.4767, -0.5923,  0.1371],
                        [ 0.2061, -1.9645,  1.1369,  1.2084],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 106 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 110 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]],
       
               [[ 0.5464, -0.1315, -0.6195],
                [ 0.2436,  1.2652,  0.5498]]]),
    x: Batch(
           c: tensor([[[ 0.0510, -1.4767, -0.5923,  0.1371],
                       [ 0.2061, -1.9645,  1.1369,  1.2084],
                       [ 1.0340, -0.8003,  0.6450,  0.1957]],
         

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 151 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498],
               [ 0.5464, -0.1315, -0.6195],
               [ 0.2436,  1.2652,  0.5498]]),
    x: Batch(
           c: tensor([[ 0.0510, -1.4767, -0.5923,  0.1371],
                      [ 0.2061, -1.9645,  1.1369,  1.2084],
                      [ 1.0340, -0.8003,  0.6450,  0.1957],
                      [ 0.0510, -1.4767, -0.5923,  0.1371],
                      [ 0.2061, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 524 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 2.82 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
